In [1]:
import random
import pickle
import re

wordnet = {}
with open("./data/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)

In [2]:
wordnet

{'호흡하': ['호흡하'],
 '물': ['물', '놈'],
 '질식하': ['질식하'],
 '전체': ['전체', '전적', '전부', '총체', '전반'],
 '한숨짓': ['한숨짓'],
 '생물': ['생물'],
 '미생물': ['미생물', '생물'],
 '헐떡이': ['헐떡이'],
 '셀': ['셀', '세포'],
 '하품하': ['하품하'],
 '인': ['인', '인간', '인물', '사람', '인류'],
 '미물': ['미물', '생물', '조수'],
 '기상하': ['기상하'],
 '자연물': ['자연물'],
 '영양': ['영양', '자양', '영양분', '양분', '영양소'],
 '물건': ['물건', '기품', '물', '물품', '물체', '놈'],
 '학식': ['학식', '학문', '인식', '알음알이', '인지', '지식', '경험', '식견'],
 '속성': ['속성'],
 '긴장하': ['긴장하'],
 '감정': ['감정', '마음', '기분'],
 '유형': ['유형',
  '폼',
  '형상',
  '체형',
  '형',
  '자세',
  '맵시',
  '형태',
  '형용',
  '복장',
  '꼴',
  '거푸집',
  '모양',
  '짝',
  '형체',
  '용모',
  '외형',
  '모양새',
  '모형'],
 '시간': ['시간', '성상', '세월'],
 '길': ['길', '과정', '경로'],
 '짓': ['짓', '법', '소위', '행위'],
 '집단': ['집단', '무리', '군', '통'],
 '상관': ['상관', '연고', '관계', '인연', '사이', '동안', '연관', '관련'],
 '재': ['재', '소장', '것', '점령', '소유물', '손', '소유', '향유', '점유', '재물'],
 '커뮤니케이션': ['커뮤니케이션', '교신', '통신'],
 '수량': ['수량', '정량', '양', '분량', '숱'],
 '속임질하': ['속임질하'],
 '중단': ['중단'],
 '

In [3]:
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText

In [7]:
get_only_hangul('이게 뭐하는 함수지요?')

'이게 뭐하는 함수지요?'

In [1]:
import random
import pickle
import re

wordnet = {}
with open("./data/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\admin\AppData\Local\Temp\ipykernel_21452\3047796988.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [2]:
EDA('와플대학은 대학 중에 최고다!')

['와플대학은 대학 중에 최고다!',
 '와플대학은 대학 중에 대 최고다!',
 '와플대학은 대학 중에 최고다!',
 '와플대학은 중에 대학 최고다!',
 '와플대학은 최고다! 중에 대학',
 '와플대학은 학 중에 최고다!',
 '와플대학은 미 중에 최고다!',
 '와플대학은 데 중에 최고다!',
 '와플대학은 대학 중에',
 '와플대학은 대학 중에 최고다!']

In [3]:
import pandas as pd

In [4]:
df = pd.read_excel('./data/최종감성사전.xlsx')
df

,word,emotion,emo_num
0,가능성이 늘어나다,만족,4
1,가능성이 있다고,기쁨,1
2,가능하다,편안,0
3,가볍고 상쾌하다,기쁨,1
4,가볍고 상쾌한,기쁨,1
...,...,...,...
27102,새해는 직무정지되서 꼴깝떠는거 안볼 줄알았더만 18연초부터 드럽게 재수없네,짜증,11
27103,그러니일본놈들이한국사람을더우습게알고함부로대하는거아닌가?,불만,17
27104,위증은 무쟈껀 제자리 화형으로 다스려야 함.,분노,6
27105,짱개와는 연을 끊는게 상책이다,분노,6


In [5]:
df_h = df[df['emotion']=='허무함']
df_h = df_h.reset_index(drop=True)
df_h

,word,emotion,emo_num
0,가치 없는,허무함,13
1,가치가 없다,허무함,13
2,가치가 없을,허무함,13
3,가치가 없이,허무함,13
4,가치없는,허무함,13
...,...,...,...
185,2년다닌 회사 때려쳤습니다,허무함,13
186,아베가 소녀상 치우라고 시키드나 이 쪽바리새끼들아 답없다 진짜 이게 나라가...,허무함,13
187,이제야... 코끼는 동물원 외에는 없다...,허무함,13
188,국민들 피땀흘린 세금이 엉뚱한 곳에 있다니..,허무함,13


In [10]:
 df_h['word'][0]

'가치 없는'

In [14]:
eda =[]
for i in df_h['word']:
    # print(i)
    a = EDA(i)
    eda.append(a)

In [16]:
eda[0]

['가치 없는',
 '가치 가 없는',
 '가치 없는',
 '가 가치 없는',
 '없는 가치',
 '가 없는',
 '가치 가 없는',
 '가치',
 '가치 없는',
 '가치 없는']

In [20]:
my_set = list(set(eda[0]))
my_set

['가치', '가 가치 없는', '가 없는', '가치 가 없는', '가치 없는', '없는 가치']

In [24]:
aug = df[(df['emotion']=='허무함') | (df['emotion']=='상처')| (df['emotion']=='당황')]
aug

,word,emotion,emo_num
2612,가짜,상처,9
2613,가치 없는,허무함,13
2614,가치가 없다,허무함,13
2615,가치가 없을,허무함,13
2616,가치가 없이,허무함,13
...,...,...,...
26385,아베가 소녀상 치우라고 시키드나 이 쪽바리새끼들아 답없다 진짜 이게 나라가...,허무함,13
26405,이제야... 코끼는 동물원 외에는 없다...,허무함,13
26762,티파니가 친일파일줄이야...,당황,8
26852,국민들 피땀흘린 세금이 엉뚱한 곳에 있다니..,허무함,13


In [28]:
eda =[]
for i in aug['word']:
    # print(i)
    a = EDA(i)
    eda.append(a)

In [30]:
eda

[['가짜', '가짜', '짜', '가짜', '가짜', '가 가짜', '가 가짜', '짜', '가 가짜', '가짜'],
 ['가 없는',
  '없는 가치',
  '가치 가 없는',
  '없는 가치',
  '가치 없는',
  '가치 가 없는',
  '치 없는',
  '가치 가 없는',
  '가 없는',
  '가치 없는'],
 ['가치가 없다',
  '가치가 없다',
  '가치가 없다',
  '없다 가치가',
  '가치가 없다',
  '가치가',
  '가치가 없다',
  '가치가 없다',
  '가치가 없다',
  '가치가 없다'],
 ['가치가 없을',
  '가치가 없을',
  '가치가 없을',
  '없을 가치가',
  '없을 가치가',
  '가치가 없을',
  '가치가 없을',
  '없을 가치가',
  '가치가 없을',
  '가치가 없을'],
 ['가치가 없이',
  '가치가 없이',
  '가치가 없이',
  '가치가 없이',
  '가치가 없이',
  '가치가 없이',
  '가치가 없이',
  '가치가 없이',
  '없이 가치가',
  '가치가 없이'],
 ['가치없는',
  '가치없는',
  '가치없는',
  '가치없는',
  '가치없는',
  '가치없는',
  '가치없는',
  '가치없는',
  '가치없는',
  '가치없는'],
 ['누르다',
  '강제로 누르다',
  '강제로 누르다',
  '강제로 누르다',
  '강제로 누르다',
  '누르다 강제로',
  '누르다',
  '누르다 강제로',
  '누르다 강제로',
  '강제로 누르다'],
 ['강제로 누름',
  '누름 강제로',
  '강제로 누름',
  '누름 강제로',
  '강제로 누름',
  '강제로 누름',
  '강제로 누름',
  '강제로 누름',
  '강제로 누름',
  '강제로 누름'],
 ['고통을 당하여',
  '당하여 고통을',
  '고통을 당하여',
  '고통을 당하여',
  '고통을 당하여',
  '고통을 당하여',
  '고통을 당하여',
  '당하여 고통을',
  '고통을 당하여